<a href="https://colab.research.google.com/github/Victor0vich/Denis/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_1_%D0%BE%D0%BA%D1%82%D1%8F%D0%B1%D1%80%D1%8F_%D0%9C%D0%B0%D1%80%D0%B0%D1%84%D0%BE%D0%BD_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE_%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2_%D0%94%D0%B5%D0%BD%D1%8C_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Схема агентов

                    

```

                      |Агент Отчета о потребностях|
      _____________________________|__________________________________________
     |                             |                                          |
     |                             |                              |Агент Презентации и ПВ|
     |                 |Агент Выявления Возражений|                           |
     |                             |                                |Агент Стилизации 2|
     |                             |                                          |
     |                            / \                                |Агент Консультант|  
     |                           /   \
     |  |Агент Выявления вопросов|   |Агент Преодоления возражений|                       
     |                 |                             |
     |         |Агент Консультант|                   |          
     |                 |                             |                         
     |         |Агент Стилизации 1|                  |
     |                 |_____________________________|    
     |                             |      
     |                    |Агент Потребностей|                         
     |_____________________________|                   
                        

```

* `Агент Выявления возражений` - этот агент выявляет возражения в сообщении клиента

* `Агент Отчета о потребностях` - этот агент создает отчет о потребностях

* `Агент Потребностей` - этот агент формулирует вопросы для выявления потребностей клиента

* `Агент Выявления вопросов` - этот агент определяет клиент размышляет/отвечает или задает вопрос. Если размышляет/отвечает, то эти размышления переделываются в форму вопроса перед подачей в `Агент Консультант`. Если клиент задает вопрос, то этот вопрос без изменений подается в `Агент Консультант`

* `Агент Закрытия возражений` - этот агент на основании выявленного возражения формулирует сообщение, закрывающее это возражение

* `Агент Консультант` - этот агент формирует ответы на основании базы знаний УИИ

* `Агент Презентации` - этот агент формирует презентацию опираясь на отчет по потребностям

* `Агент Стилизации 1` - этот агент "причесывает" итоговый ответ после `Агент Консультант`, убирая лишнее

* `Агент Стилизации 2` - этот агент "причесывает" итоговый ответ в стиле продажи после `Агент Презентации`, убирая лишнее







In [ ]:
#@title Установка и импорт библиотек

!pip install tiktoken langchain==0.0.330 openai==0.28.1 faiss-cpu

import textwrap
import zipfile

# Модуль для работы с функциями, связанными с паролями
import getpass

# Модуль для загрузки файлов из Google Drive
import gdown


# Модуль для работы с файловой системой и операционной системой
import os

# Модуль для работы с регулярными выражениями
import re

# Библиотека для работы с API OpenAI
import openai

# Модуль для создания глубоких копий объектов
import copy

# Импортируем стандартный модуль time, который предоставляет функции для работы с временем
import time

# Модуль для работы с токенизацией текста
import tiktoken

# Импортируем загрузчик текстовых документов из langchain
from langchain.document_loaders import TextLoader

# Импортируем прогресс-бар для Jupyter Notebook
from tqdm import tqdm_notebook as tqdm

# Модуль для работы с сериализацией и десериализацией объектов
import pickle

# Импортируем модель OpenAI из langchain
from langchain.llms import OpenAI

# Импортируем класс Document из langchain для работы с документами
from langchain.docstore.document import Document

# Библиотека для выполнения HTTP-запросов
import requests

# Импортируем класс для создания эмбеддингов с помощью OpenAI из langchain
from langchain.embeddings.openai import OpenAIEmbeddings

# Импортируем класс FAISS для работы с векторными индексами из langchain
from langchain.vectorstores import FAISS

# Импортируем класс для рекурсивного разделения текста на фрагменты из langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Игнорирование предупреждений
import warnings
warnings.filterwarnings("ignore")

# Настройка логирования для модуля text_splitter из langchain
import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
# Получение ключа API OpenAI от пользователя
# Функция getpass.getpass() запрашивает ввод пароля или ключа без отображения ввода на экране
openai_key = getpass.getpass("OpenAI API Key:")

# Установка ключа API в переменную окружения
os.environ["OPENAI_API_KEY"] = openai_key

# Установка ключа API для использования библиотекой openai
openai.api_key = openai_key

OpenAI API Key:··········


## Агент Консультант для ответов на вопросы по базе знаний

In [ ]:
#@title Загружаем базу знаний

# Указание URL-адреса файла и путь для скачивания
url = "https://drive.google.com/uc?id=116pmFdDoPxmYRy-K1FPyUOLYQ7g6ftmq"
output_zip = "index.zip"  # Имя архива после скачивания

# Скачивание файла
gdown.download(url, output=output_zip, quiet=False)

# Создание папки для распаковки (если не существует)
output_folder = "HistoryUII"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Разархивирование содержимого в папку UII
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall(output_folder)


def create_search_index(db_path, course):
    # Путь к файлу index.faiss
    faiss_file = os.path.join(db_path, course, 'index.faiss')
    #print(faiss_file)

    # Загрузка базы данных FAISS
    db = FAISS.load_local('/content/HistoryUII/', OpenAIEmbeddings())

    return db

# У вас только один курс
course = 'HistoryUII'
faiss_db_dir = "/content/"

# Загружаем индекс
search_index = create_search_index(faiss_db_dir, course)

db = search_index

Downloading...
From: https://drive.google.com/uc?id=116pmFdDoPxmYRy-K1FPyUOLYQ7g6ftmq
To: /content/index.zip
100%|██████████| 8.21M/8.21M [00:00<00:00, 58.8MB/s]


In [ ]:
#@title Функция для получения ответа на вопрос на основе заданной темы и поискового индекса
def answer_kons(system, instruction, topic, search_index, summary_history, temp=1, verbose=0, k=8, model="gpt-3.5-turbo-16k"):
    """
    Функция для получения ответа на вопрос на основе заданной темы и поискового индекса.

    Параметры:
    system (str): Промт.
    instruction (str): Инструкция для пользователя.
    topic (str): Тема или вопрос, на который нужно ответить.
    search_index (object): Векторная база.
    temp (float): Температура для генерации ответов модели (по умолчанию 1).
    verbose (int): Флаг для включения/выключения подробного вывода (по умолчанию 0).
    k (int): Количество похожих документов для поиска (по умолчанию 8).
    model (str): Имя модели, используемой для генерации ответов (по умолчанию "gpt-3.5-turbo-16k").
    """

    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n ========\n')

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nХронология предыдущих сообщений диалога: {summary_history}\n\nОтвет:"}
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI
    completion = openai.ChatCompletion.create(
        model=model,  # Имя модели
        messages=messages,  # Сообщения для модели
        temperature=temp  # Температура генерации
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n =========================================== ')


    # Получение ответа от модели
    answer = completion.choices[0].message.content
    #print()
    #print()
    #print("Полный ответ от GPT:")
    #print(completion)
    print('\n==================')
    print("Агент консультант:")
    # Форматирование ответа для удобства чтения
    formatted_answer = textwrap.fill(answer, width=120)
    print(formatted_answer)
    #print(f"Количество использованных токенов на запрос: {completion['usage']['prompt_tokens']}")
    #print(f"Количество использованных токенов на ответ: {completion['usage']['completion_tokens']}")
    #print(f"Общее количество использованных токенов (вопрос-ответ): {completion['usage']['total_tokens']}")
    #print()
    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    print('ЦЕНА запроса:', price, ' $')
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer


def answer_kons_d(system, instruction, topic, search_index, summary_history, temp=1, verbose=0, k=8, model="gpt-3.5-turbo-16k"):
    """
    Функция для получения ответа на вопрос на основе заданной темы и поискового индекса.

    Параметры:
    system (str): Промт.
    instruction (str): Инструкция для пользователя.
    topic (str): Тема или вопрос, на который нужно ответить.
    search_index (object): Векторная база.
    temp (float): Температура для генерации ответов модели (по умолчанию 1).
    verbose (int): Флаг для включения/выключения подробного вывода (по умолчанию 0).
    k (int): Количество похожих документов для поиска (по умолчанию 8).
    model (str): Имя модели, используемой для генерации ответов (по умолчанию "gpt-3.5-turbo-16k").
    """

    # Поиск документов, похожих на заданную тему/вопрос
    docs = search_index.similarity_search(topic, k=k)

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n ========\n')

    # Создание содержимого сообщения, объединяя текст найденных документов
    message_content = re.sub(r'\r\n', ' ', '\n '.join([f'\n--------------------\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    # Если включен подробный вывод, напечатать содержимое сообщения
    if verbose: print('Чанки :\n ======================================== \n', message_content)

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instruction}.\n{message_content}.\n\nВопрос:\n{topic}\n\nХронология предыдущих сообщений диалога: {summary_history}\n\nОтвет:"}
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI
    completion = openai.ChatCompletion.create(
        model=model,  # Имя модели
        messages=messages,  # Сообщения для модели
        temperature=temp  # Температура генерации
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Если включен подробный вывод, напечатать разделитель
    #if verbose: print('\n =========================================== ')


    # Получение ответа от модели
    answer = completion.choices[0].message.content
    #print()
    #print()
    #print("Полный ответ от GPT:")
    #print(completion)
    print('\n==================')
    print("Агент консультант:")
    # Форматирование ответа для удобства чтения
    formatted_answer = textwrap.fill(answer, width=120)
    print(formatted_answer)
    #print(f"Количество использованных токенов на запрос: {completion['usage']['prompt_tokens']}")
    #print(f"Количество использованных токенов на ответ: {completion['usage']['completion_tokens']}")
    #print(f"Общее количество использованных токенов (вопрос-ответ): {completion['usage']['total_tokens']}")
    #print()
    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    print('ЦЕНА запроса:', price, ' $')
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer


Промт и инструкция

In [ ]:
# Подгружаем промт
system = ''' # Ты профессиональный помощник в чате компании Университета Искусственного интеллекта.


## 1. Общие обязанности и цели - Компания продает курсы по AI. У компании есть большой документ со всеми материалами
о продуктах компании. - Твоя обязанность: Дать Клиенту краткий корректный ответ на русском языке в чате, опираясь на отрывки
из этого документа. - Твоя цель: Отвечать максимально кратко и точно по документу, не придумывать ничего от себя.


## 2. Ограничения в общении - Запрещено общаться на стороннюю тему. Если Клиент задает стороннюю тему, спрашивает не
по теме искусственного интеллекта и программирования, не по материалам и продуктам Компании, ты категорически отказываешься
отвечать.


## 3. Секретность документа
- Запрещено упоминать в ответе, что ты анализировал отрывки документов и брал оттуда информацию.


## Обрати внимание: курс по созданию нейроконсультантов - это курс "Нейро-сотрудники",
## жпт - это chatGPT, если клиент употребляет слово жпт, понимай это слово как chatGPT '''

# Инструкция для модели о том, как использовать контекст для ответа на вопрос
instruction = '''Проанализируй предыдущий диалог чтобы написать свой ответ последовательным и логичным.
Категорически запрещено повторяться и здороваться.
Используйте следующие фрагменты контекста, чтобы ответить на вопрос в конце.
Стилистика ответа должна быть поддерживающей беседу в контексте важности и полезности изучения искусственного интеллекта и программирования.
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумывать ответ.
Тебе запрещено продавать, предлагать курсы. Запрещено спрашивать клиента что его еще интересует.'''

Параметры для запроса

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose = 1

# Имя модели, используемой для генерации ответов
model = "gpt-4o"

# История диалога

history = [""]

Запрос

In [ ]:
# Тема или вопрос, на который нужно получить ответ
topic = "Какая стоимость курса ChatGPT Professional light?"

In [ ]:
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);

Чанки :
 
--------------------
#### ChatGPT Professioal LIGHT+
 Стоимость тарифа составляет: 179 900
 Продолжительность курса: 11 месяцев
В тариф входит:
    - поддержка куратора в течение всего курса
    - Курс нейро-сотрудники (6 занятий)
    - Курс Python разработчик (36 занятий)
    - 2 новых курса на выбор (10 занятий)
    - Гарантия трудоустройства
Количество мест на тарифе ограничено. УИИ набирает только 10 студентов на каждый поток тарифа

#### ChatGPT Professional PRO+
      Стоимость тарифа составляет: 239 900 рублей
      Продолжительность курса: 11 месяцев 
Иерархия фрагмента текста в документе: # Университет Искусственного Интеллекта ## Формат курса СhatGPT Professional в записи и дата старта курса ### Стоимость курса ChatGPT Professional в записи и тарифы курса: #### ChatGPT Professional PRO+

 
--------------------
Тариф “AI под ключ” идеально подходит для компаний с проектом или для тех, кто предпочитает проходить курс индивидуально.
Так же он подходит студентам, которы

Запрос

In [ ]:
# Тема или вопрос, на который нужно получить ответ
topic = "Что такое выкуп стоимости?"

In [ ]:
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);

Чанки :
 
--------------------
### Условия выкупа стоимости прохождения курса
Студентам предоставляется возможность вернуть всю стоимость курса и трудоустроиться в УИИ после прохождения программы. 
Гарантия трудоустройства прописана в договоре. Возможность трудоустроиться предоставляется на следующий день после защиты диплома. Таким образом, студент возвращает всю стоимость курса в виде заработной платы. На выкуп стоимости открывается 10 мест. Более 150 студентов уже вернули стоимость своего курса.
Все условия выкупа прописаны в отдельном дополнительном соглашении.
Чтобы ознакомиться с дополнительным соглашением на выкуп стоимости курса от УИИ, необходимо обратиться к менеджеру УИИ.
Необходимо в полном объеме программу курса, в том числе выполнить все без исключения домашние задания продвинутого (Pro) уровня сложности к каждому занятию не более чем за 10 месяцев пользованием платформой или за определенное сторонами Договора количество занятий (промежуточные аттестации), успешно защитит

Запрос

In [ ]:
# Тема или вопрос, на который нужно получить ответ
topic = "Сколько времени необходимо для обучения?"

In [ ]:
answer_kons(system, instruction, topic, db, history, temperature, verbose, relevant_chanks, model);

Чанки :
 
--------------------
Дефицит времени достаточно распространенная проблема в современном мире. Известно, что дефицит времени у людей встречается чаще, чем дефицит денег.
Образовательный процесс в УИИ выстроен таким образом, чтобы проходить курс в удобное время, не отвлекаясь от профессиональных и домашних дел. Студенту потребуется минимум 4 часа, а лучше 10 часов в неделю, которые включают в себя 2 часа теории, 2 часа практических занятий и 6 часов выполнения домашнего задания или стажировки. Все, что необходимо от студента - желание освоить новые знания, так как проходить курс возможно в обеденный перерыв на работе, в дороге, в очереди. Видеоуроки открываются в том месте, где студент остановил просмотр. По подсчетам УИИ, курс занимает всего 7 % времени в неделю.

В среднем, распределение времени человека в возрасте 35-40 лет происходит так:
35% - работа
37% - семья и друзья
13% - отдых (кино, кафе, отпуск)
8% - хобби (занятия творчеством,спортом)
7% - прохождение курсов (10 ч

## Агент Отчета о потребностях

In [ ]:
#@title Функция для отчета о потребностях
def user_question_router(system, instructions, topic, temp=0, verbose=0, model='gpt-3.5-turbo-1106'):
    #if verbose: print('\n==================\n')
    #if verbose: print('Саммари диалога:\n==================\n',
                      #topic)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nВопрос менеджера и ответ клиента:\n{topic}.
                                      \n\nОтвет: '''}
    ]

    start_time = time.time()  # Начало отсчета времени

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print('\n==================')
    if verbose: print(f'Агент Отчета о потребностях:\n',
                      f'{formatted_answer}')

    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    print('ЦЕНА запроса:', price, ' $')
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer

# Функция суммарной потребности
def user_potr(potr_history, temp=0, verbose=0, model= "gpt-4o"):
    resulting_string = ", ".join(potr_history)
    completion_prez = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": """Вы лучший менеджер по продажам.
Вы понимаете какие потребности клиента надо выявить, чтобы полностью понять желания и боли клиента,
которые можно удовлетворить при помощи обучения искусственному интеллекту и программированию.
Вы знаете, что важно выявить есть ли у клиента такие потребности: в трудоустройстве, в росте дохода,
в возможности работать удаленно, развиваться в сфере IT, сменить деятельность, в карьерном росте, во фрилансе,
в улучшении текущей позиции в компании, в развитии своего проекта (если он есть), в получении прибыли от своего
проекта (если он есть), в увекательном хобби, в личностном развитии и другие."""},
        {"role": "user", "content": f''' На основании Отчета по потребностям отпредели основную потребность клиента. В ответе четко сформулируй только эту потребность в виде темы для статьи.

         Отчет по потребностям: {resulting_string}'''}],
        temperature=0
    )
    answer_prez = completion_prez.choices[0].message.content
    #print(answer_prez)
    return answer_prez

Промт и инструкция

In [ ]:
system_prompt_router = '''
Ты лучший специалист отдела продаж. Ты продаешь курсы обучения искусственному интеллекту и программированию.
Ты знаешь, что Потребность - это то, что клиент хочет или что ему нравится,
и что повлияет на приобретение им курсов обучения.
Ты очень хорошо умеешь выявлять в Вопросе менеджера и ответе клиента потребности клиента.
Вы знаете, что важными потребностями клиента являются потребности: в трудоустройстве, в росте дохода,
в возможности работать удаленно, развиваться в сфере IT, сменить деятельность, в карьерном росте, во фрилансе,
в улучшении текущей позиции в компании, в развитии своего проекта (если он есть), в получении прибыли от своего
проекта (если он есть), в увекательном хобби, в личностном развитии и другие.
Ты всегда очень строго следуешь порядку отчета.
'''
instructions_router = '''
Давай действовать последовательно:
Проанализируй Вопрос менеджера и ответ клиента, выяви какие высказаны потребности клиента.
Ничего не придумывай от себя.
Порядок отчета: предоставь только название потребности с формулировкой клиента через двоеточие.'''


Параметры запроса

In [ ]:
model_router = 'gpt-4o' # Имя модели, используемой для генерации ответов

temperature_router = 0 # Задаем температуру генерации ответа

verbose_router = 1 # Флаг для включения/выключения вывода ответа модели

topic = "я не хочу работать на фрилансе" # Тема или вопрос, на который нужно получить ответ

Запрос

In [ ]:
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);


Агент Отчета о потребностях:
 Потребность: стабильная работа в компании.
ЦЕНА запроса: 0.00111  $
Время ответа: 0.37 секунд


Запрос

In [ ]:
topic = "мне надо проект" # Тема или вопрос, на который нужно получить ответ
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);


Агент Отчета о потребностях:
 Потребность: проект.
ЦЕНА запроса: 0.00102  $
Время ответа: 0.63 секунд


Запрос

In [ ]:
topic = "пока точно не знаю. хочу сначала пройти курсы" # Тема или вопрос, на который нужно получить ответ
user_question_router(system_prompt_router, instructions_router, topic, temperature_router, verbose_router, model_router);


Агент Отчета о потребностях:
 Потребность: пройти курсы.
ЦЕНА запроса: 0.00109  $
Время ответа: 0.30 секунд


## Агент Выявления возражений

In [ ]:
#@title Функция для выявления возражений
def user_objection_router(system, instructions, topic, temp=0, verbose=0, model='gpt-3.5-turbo-1106'):
    #if verbose: print('\n==================\n')
    #if verbose: print('Саммари диалога:\n==================\n',
                      #topic)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nВопрос менеджера и ответ клиента:\n{topic}.
                                      \n\nОтвет: '''}
    ]

    start_time = time.time()  # Начало отсчета времени

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print('\n==================')
    if verbose: print(f'Агент Отчета о возражениях:\n',
                      f'{formatted_answer}')

    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    print('ЦЕНА запроса:', price, ' $')
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer


Промт и инструкция

In [ ]:
system_prompt_objection = '''
Ты - лучший специалист отдела продаж, специализирующийся на продаже курсов обучения искусственному интеллекту.
Ты знаешь, что открытое возражение  - это явное заявление со стороны клиента о том, что его не устраивает и
что может быть преградой для приобретения курсов обучения, а просьбы клинета не являются возражениями.
Твоя задача - тщательно выявлять открытые возражения в Вопросе клиента.
Ты всегда строго следуешь порядку отчета.
'''
instructions_objection = '''
Давай действовать последовательно:
Проанализируй Вопрос клиента и выдели открытые возражения, которые он высказывает (если они есть).
Примеры возражений могут включать в себя: "дорого", "мне не хватит знаний/опыта",
"мне не хватит времени на учебу", "слышал плохие отзывы", "наполнение", "все только обещают" и т.п.
В своем ответе укажи все выделенные открытые возражения клиента, соблюдая ту же последовательность, в какой они были
высказаны в его вопросе. Ничего не придумывай от себя: если возражений  не найдено, то в своем ответе напиши "-".
Порядок отчета: в свой ответ включи только список выделенных открытых возражений клиента (или  "-") в виде строки с разделителями (запятые).'''


Параметры запроса

In [ ]:
model_objection = 'gpt-4o' # Имя модели, используемой для генерации ответов

temperature_objection = 0 # Задаем температуру генерации ответа

verbose_objection = 1 # Флаг для включения/выключения вывода ответа модели

topic = "Я хочу начать обучение, но для меня это дорого" # Тема или вопрос, на который нужно получить ответ

Запрос

In [ ]:
user_objection_router(system_prompt_objection, instructions_objection, topic, temperature_objection, verbose_objection, model_objection);


Агент Отчета о возражениях:
 дорого
ЦЕНА запроса: 0.00176  $
Время ответа: 0.31 секунд


Запрос

In [ ]:
topic = "я не смогу работать на фрилансе" # Тема или вопрос, на который нужно получить ответ
user_objection_router(system_prompt_objection, instructions_objection, topic, temperature_objection, verbose_objection, model_objection);


Агент Отчета о возражениях:
 мне не хватит знаний/опыта
ЦЕНА запроса: 0.00186  $
Время ответа: 0.47 секунд


Запрос

In [ ]:
topic = "сколько стоит пройти курс" # Тема или вопрос, на который нужно получить ответ
user_objection_router(system_prompt_objection, instructions_objection, topic, temperature_objection, verbose_objection, model_objection);


Агент Отчета о возражениях:
 -
ЦЕНА запроса: 0.00172  $
Время ответа: 0.25 секунд


## Агент Закрытия возражений

In [ ]:
#@title Функция для закрытия возражений
def user_objection_close(system, instructions, topic, temp=0, verbose=0, k=3, model= "gpt-4o"):
    #if verbose: print('\n==================\n')
    #if verbose: print('Саммари диалога:\n==================\n',
                      #topic)
   # knowledge_base = search_index.similarity_search(topic, k=k)
    #docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n==================\n' + doc.page_content + '\n' for doc in knowledge_base]))

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nВозражение клиента:\n{topic}.
                                      '''}
    ]

    start_time = time.time()  # Начало отсчета времени

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print('\n==================')
    if verbose: print(f'Агент Закрытия возражений:\n',
                      f'{formatted_answer}')
    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    print('ЦЕНА запроса:', price, ' $')
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer


Промт и инструкция

In [ ]:
system_prompt_objection_close = '''
Вы лучший специалист по отработке возражений клиента. Вы работаете в Университете (УИИ), который продает курсы обучения нейронным сетям и программированию.
Вы знаете, что отработка возражения - это процесс убеждения с помощью аргументов,
точных фактов и психологических приемов, которые помогают завуалированно донести до клиента мысль, что курсы ему нужны.
Вы знаете, что отработка возражений не имеет ничего общего с давлением: продавец не манипулирует, а показывает как удовлетворить потребности клинета с помощью продукта.
Вы всегда последовательно, шаг за шагом и убедительно отрабатываете возражения клиента.
Вы всегда очень строго следуете порядку отчета.
'''
instructions_objection_close = '''
Пожалуйста, проанализируйте Возражение клиента и напишите убедительную и качественную отработку возражения клиента.
Основные возражения и их закрытие:
- Сколько это стоит? Это платно? Курсы это дорого - закрытие возражения: курсы стоят от 150 тысяч, это очень доступно по цене, есть рассрочки от банкой и платёж всего от 4719р в месяц
- Я хочу только вебинар, Я хочу только бесплатное - закрытие возражения: да, конечно, вы можете сходить на вебинар, но менеджер просто даст вам больше информации и вы сможете потом взвешенно принимать решение о прохождении курса
- Зачем мне общаться с человеком - закрытие возражения: он сможет вам значительно более подробно рассказать о курсе и вы сможете потом взвешенно принимать решение о прохождении курса
- Я не программист, гуманитарий, у меня не получится - закрытие возражения: 80% наши студентов не программисты, наши курсы построены полностью для не программистов, у вас точно получится
- У меня нет времени на обучение - закрытие возражения: обучение занимает всего несколько часов в неделю и вы можете заниматься в любой момент времени
- Мне уже много лет, у меня не получится - закрытие возражения: нашему самому старшему выпускнику 82 года, средний возраст студентов 35-40 лет, вы точно можете изучить AI в вашем возрасте
- Чем вы лучше конкурентов - закрытие возражения: мы единственные, кто преподаёт только AI, мы в 3 раза обгоняем всех по количеству контента по AI, у нас единственных есть стажировки в топовых компаниях РФ и мы единственные, у кого преподают и проверяют домашние задание нейросети.
Порядок отчета: в свой ответ выведите только отработку возражения.'''


Параметры запроса

In [ ]:
model_objection_close = 'gpt-4o' # Имя модели, используемой для генерации ответов

temperature_objection_close = 0 # Задаем температуру генерации ответа

verbose_objection_close = 1 # Флаг для включения/выключения вывода ответа модели

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

topic = "дорого" # Тема или вопрос, на который нужно получить ответ

Запрос

In [ ]:
user_objection_close(system_prompt_objection_close, instructions_objection_close, topic, temperature_objection_close, verbose_objection_close, relevant_chanks, model_objection_close);


Агент Закрытия возражений:
 Понимаю ваше беспокойство по поводу стоимости. Наши курсы стоят от 150 тысяч рублей, и это действительно доступная цена,
учитывая качество и объем предоставляемого материала. Более того, у нас есть возможность рассрочки от банков, что
позволяет вам платить всего от 4719 рублей в месяц. Это делает обучение доступным для большинства людей, даже если у вас
ограниченный бюджет. Инвестируя в эти курсы, вы получаете уникальные знания и навыки, которые могут значительно повысить
вашу конкурентоспособность на рынке труда и открыть новые карьерные возможности.
ЦЕНА запроса: 0.00479  $
Время ответа: 1.46 секунд


Запрос

In [ ]:
topic = "мне не хватит знаний/опыта" # Тема или вопрос, на который нужно получить ответ
user_objection_close(system_prompt_objection_close, instructions_objection_close, topic, temperature_objection_close, verbose_objection_close, relevant_chanks, model_objection_close);


Агент Закрытия возражений:
 Понимаю ваше беспокойство по поводу недостатка знаний и опыта. На самом деле, это очень распространенное опасение среди
наших студентов. Позвольте мне объяснить, почему это не должно вас останавливать.  Наши курсы специально разработаны для
людей с разным уровнем подготовки, включая тех, кто никогда не сталкивался с программированием или нейронными сетями.
Более того, 80% наших студентов не имеют программного опыта, и наши курсы построены таким образом, чтобы они могли
успешно освоить материал.  Мы начинаем с самых основ и постепенно усложняем материал, чтобы вы могли уверенно двигаться
вперед. Наши преподаватели и менторы всегда готовы помочь вам на каждом этапе обучения, отвечая на вопросы и
предоставляя дополнительную поддержку.  Кроме того, у нас есть уникальная система проверки домашних заданий с
использованием нейросетей, что позволяет вам получать мгновенную обратную связь и улучшать свои навыки в реальном
времени.  Таким образом, даже если у вас нет

## Агент Потребностей

In [ ]:
#@title Функция для генерации вопроса пользователю на основе истории чата и текущего вопроса
# Функция для генерации отчета по истории чата
def generate_summary_report(history_chat):
    report = []
    for i, message in enumerate(history_chat, start=1):
        report.append(f"{i}. {message}")  # Нумеруем и добавляем каждое сообщение в отчет
    return "\n".join(report)  # Объединяем все сообщения в одну строку


# Функция для генерации вопроса пользователю на основе истории чата и текущего вопроса
def spez_user_question(system, instructions, needs, history_chat, temp=0, verbose=0, model="gpt-3.5-turbo-16k"):
    summary_history = generate_summary_report(history_chat)  # Генерация отчета по истории чата
    messages = [
        {"role": "system", "content": system},  # Добавляем системное сообщение
        {"role": "user", "content": f'''{instructions}

         Список потребностей: {needs}

         Хронология предыдущих сообщений диалога: {summary_history}'''}  # Добавляем инструкции и контекст
    ]

    # if verbose: print('\n==================\n')
    # if verbose: print(f'Вопрос клиента:', question)
    # if verbose: print('Саммари диалога:\n==================\n',
    #                      summary_history)
    # print("messages", messages)

    start_time = time.time()  # Начало отсчета времени

    # Запрос к модели OpenAI для генерации ответа
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # Извлекаем ответ из ответа модели
    answer = completion.choices[0].message.content
    try:
        # Пытаемся разделить ответ по ': ' и взять вторую часть
        answer = answer.split(': ')[1] + ' '
    except:
        # Если не удалось, используем оригинальный ответ
        answer = answer
    # Убираем возможные служебные символы в начале строки
    answer = answer.lstrip('#3')

    # if verbose: print(f'\n==================')
    # if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print('\n==================')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print(f'Вопрос от Агента Потребностей:\n', f'{formatted_answer}')
    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    if verbose: print('ЦЕНА запроса:', price, ' $')
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer  # Возвращаем ответ



Промт и инструкция

In [ ]:
# Системное сообщение для модели, задающее её поведение и контекст работы
system_prompt_potr = '''
Вы лучший менеджер по продажам. Вы работаете в в Университете Искусственного интеллекта (сокр УИИ),
который продает курсы обучения нейронным сетям и языку программирования python.
Вы неплохо разбираетесь в нейронных сетях и программировании в целом и очень хорошо знаете, какие курсы и программы обучения предоставляет УИИ.
Вы понимаете какие потребности клиента надо выявить, чтобы полностью понять желания и боли клиента,
которые можно удовлетворить при помощи обучения в области искусственного интеллекта и программирования.
Вы знаете, что важно выявить есть ли у клиента такие потребности: в трудоустройстве, в росте дохода,
в возможности работать удаленно, развиваться в сфере IT, сменить деятельность, в карьерном росте, во фрилансе,
в улучшении текущей позиции в компании, в развитии своего проекта (если он есть), в получении прибыли от своего
проекта (если он есть), в увекательном хобби, в личностном развитии и другие.
Вы понимаете, что нужно аккуратно и ненавязчиво выявить несколько разных потребностей и составляете свой ответ с этой целью.
Ваша задача: сформулировать вопрос клиенту, который поможет качественно выявить его потребности.
Вы всегда очень строго следуете порядку отчета.
'''

# Инструкции для модели, описывающие шаги, которые она должна выполнить
instructions__potr = '''
Давайте действовать по шагам:
#Шаг1: Проанализируйте диалог и Список потребностей;
#Шаг2: Предположите одну другую потребность, которой нет среди выявленных потребностей в Шаг1;
#Шаг3: Учитывая, что выявление потребностей должно провоцировать дальнейший диалог и заинтересовывать клиента,
для этой потребности из Шаг2 напишите один вопрос для качественного выявления этой потребности. Ничего, кроме вопроса не пишите и не объясняйте.
Порядок отчета: В свой ответ включите только один вопрос из Шаг3, ничего кроме вопроса выводить не нужно.
'''

Параметры запроса

In [ ]:
temp = 0.1  # Температура генерации ответов модели, влияет на креативность и разнообразие ответов

verbose = 1 # Флаг для включения/выключения вывода ответа модели

model_potr = 'gpt-4o' # Имя модели, используемой для генерации ответов

history_chat = ["""Менеджер: Здравствуйте, я нейро-консультант УИИ. Я знаю все об УИИ и готов помочь с любыми вопросами о нем.
К нам часто обращаются, чтобы сменить профессию или сделать проект по ИИ. Почему Вы решили обратиться к нам?""",
"""Клиент: хочу работать на фрилансе"""] # История диалога

needs = ["Фриланс: работать на фрилансе"] # Отчет о потребностях

Запрос

In [ ]:
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);


Вопрос от Агента Потребностей:
 Какие навыки или знания, по вашему мнению, помогут вам стать успешным фрилансером?
ЦЕНА запроса: 0.00306  $
Время ответа: 0.39 секунд


Параметры запроса

In [ ]:
history_chat = ["""Менеджер: Здравствуйте, я нейро-консультант УИИ. Я знаю все об УИИ и готов помочь с любыми вопросами о нем.
К нам часто обращаются, чтобы сменить профессию или сделать проект по ИИ. Почему Вы решили обратиться к нам?""",
"""Клиент: Мне нужно на работе сделать проект"""] # История диалога

needs = ["Проект: Мне нужно на работе сделать проект"] # Отчет о потребностях

Запрос

In [ ]:
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);


Вопрос от Агента Потребностей:
 Какой результат вы хотите достичь с этим проектом, и как это повлияет на вашу карьеру?
ЦЕНА запроса: 0.00308  $
Время ответа: 0.57 секунд


Параметры запроса

In [ ]:
history_chat = ["""Менеджер: Здравствуйте, я нейро-консультант УИИ. Я знаю все об УИИ и готов помочь с любыми вопросами о нем.
К нам часто обращаются, чтобы сменить профессию или сделать проект по ИИ. Почему Вы решили обратиться к нам?""",
"""Клиент: хочу сменить профессию"""] # История диалога

needs = ["Потребность: сменить профессию."] # Отчет о потребностях

Запрос

In [ ]:
spez_user_question(system_prompt_potr, instructions__potr, needs, history_chat, temp, verbose, model_potr);


Вопрос от Агента Потребностей:
 Какие цели вы ставите перед собой в новой профессии?
ЦЕНА запроса: 0.00292  $
Время ответа: 0.33 секунд


## Агент Выявления вопросов

In [ ]:
#@title Функция для выявления вопросов
def user_question(system, instructions, topic, temp=0, verbose=0, model='gpt-3.5-turbo-1106'):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nСообщение клиента:\n{topic}
                                      \n\nОтвет: '''}
    ]

    start_time = time.time()  # Начало отсчета времени

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print('\n==================')
    if verbose: print(f'Ответ Агента по выявлению вопросов:\n',
                      f'{formatted_answer}')

    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    if verbose: print('ЦЕНА запроса:', price, ' $')
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    return answer


Промт и инструкция

In [ ]:
system_prompt_question ='''
Ты идеально справляешься со своей задачей: ты отлично определяешь вопрос клиента.
Проверь есть ли в сообщении клиента какой-нибудь вопрос.
Если вопрос есть, то напиши сообщение клиента без изменений.
Если вопроса нет, а есть общие рассуждения, то сформулируй на основании сообщения клиента вопрос.

Ты всегда очень строго следуешь требованиям к порядку отчета.'''
instructions_question = '''
Пожалуйста, будем действовать по шагам:
#Шаг1: проанализируйте Сообщение клиента чтобы быть в контексте;
#Шаг2: опираясь на анализ Шаг1 сформулируй Вопрос клиента.
Отвечай, пожалуйста, точно, и ничего не придумывай от себя.
Порядок отчета: напиши только вопрос клиента.'''

Параметры модели

In [ ]:
model_question = 'gpt-4o' # Имя модели, используемой для генерации ответов

temperature_question = 0 # Задаем температуру генерации ответа

verbose_question = 1 # Флаг для включения/выключения вывода ответа модели

topic = "Что такое выкуп стоимости" # Тема или вопрос, на который нужно получить ответ

Запрос

In [ ]:
user_question(system_prompt_question, instructions_question, topic, temperature_question, verbose_question, model_question);


Ответ Агента по выявлению вопросов:
 Что такое выкуп стоимости
ЦЕНА запроса: 0.00105  $
Время ответа: 0.27 секунд


Запрос

In [ ]:
topic = "я сомневаюсь в своих силах, у меня нет опыта в реальных проектах"
user_question(system_prompt_question, instructions_question, topic, temperature_question, verbose_question, model_question);


Ответ Агента по выявлению вопросов:
 Как мне преодолеть сомнения в своих силах и начать работать над реальными проектами?
ЦЕНА запроса: 0.00134  $
Время ответа: 0.39 секунд


## Агент Презентации

In [ ]:
#@title Функция для презентации продукта

def prez_user_question(system, instructions, potr_history, sp_vozr, temp=0, verbose=0, k=3, model= "gpt-4o"):

    #knowledge_base = search_index.similarity_search(potr_history, k=k)
    #docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n==================\n' + doc.page_content + '\n' for doc in knowledge_base]))

    vozr = generate_summary_report(sp_vozr)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}

         Отчет по потребностям: {potr_history}

         Список возражений клиента: {vozr}'''}
    ]
    #if verbose: print('\n==================\n')
    #if verbose: print('Потребности:\n==================\n',
                         #potr_history)
    #if verbose: print(f'Список возражений клиента:\n==================\n', vozr)

    start_time = time.time()  # Начало отсчета времени

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    answer = completion.choices[0].message.content
    #try:
      #answer = answer.split(': ')[1]+ ' '
    #except:
      #answer = answer
    #answer = answer.lstrip('#3')
    #if verbose: print(f'\n==================')
    #if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    formatted_answer = textwrap.fill(answer, width=120)
    if verbose: print('\n==================')
    if verbose: print(f'Текст для стилизации:\n',
                      f'{formatted_answer}')
    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    if verbose: print('ЦЕНА запроса:', price, ' $')
    if verbose: print(f"Время ответа: {elapsed_time2:.2f} секунд")

    return answer


Промт и инструкция

In [ ]:
system_prompt_prez = '''Вы - лучший специалист по презентации продукта и компании Университет искусственного интеллекта (сокр. УИИ),
занимающейся продажей обучения. Ваш стиль общения деловой и очень краткий.
Ваша цель: сделать краткую убедительную и качественную презентацию по потребностям клиента (запрос может быть о курсах/программах обучения, стоимости, об УИИ).
Ваши презентации всегда основаны на потребностях и желаниях клиента.
Не упоминай явно потребности клиента.
Вы никогда не используете шаблонный скриптовый вариант презентации, всегда делаете её в неформальной форме.
'''

instructions_prez = '''
Сделайте максимально краткую убедительную и качественную презентацию, учитывая Отчет по потребностям и Список возражений клиента.
При ответе опирайся на эту информацию:
Основные выгоды для человека:
- Трудоустройство - человек хочет трудоустроиться на постоянную работу в компанию и стать AI разработчиком. Главные преимущества - зарплата 70-120 тысяч на старте когда junior, 300-500 тысяч через 3-4 года для senior, 75% вакансий удалённые и со свободным графиком, очень много творчества и высокая востребованность на рынке
- Работа на фрилансе и создание проектов на заказ - много проектов на рынке, цена проекта от 100 тысяч до 1.5млн рублей, можно не уходить с текущей работы
- Создание собственного AI проекта - упрощение своей работы, повышение эффективности для компании, повышение в должности и зарплате на 30-100%
Тезисы о рынке AI
- В России не хватает 10.000 AI разработчиков
- У нас в работе 219 вакансий и только 42 выпускника на трудоустройство
- Рост ниши AI - 38% в год, самый быстрый рост в мире

Тезисы о курсе
- 6 месяцев курса
- 4 стажировки на реальных проектах в крупных компаниях
- До 60 zoom консультаций с кураторами
- Гарантия трудоустройства по договору
- Возврат всей стоимости если не трудоустроим
- Помощь с созданием собственного AI проекта
- Нейро-куратор - отвечает на ваши вопросы 24/7 на 8 секунд с рентингом 9.6 из 10
- Нейро-проверка домашек - проверяет домашки с разбором на 1-2 страницы за 10с 24/7
- Нейро-репетитор - всегда с вами в занятиях, помогает на каждом шаге

Тезисы о компании
- Единственный преподаём только AI
- Сама большая база AI контента - более 340 занятий
- Стажировки в компаниях, работаем с: РЖД, Совкомбанк, Росталеком, Самолет, KIA, Wildberries и другими
- Собственное IA HR агентство AI hunter, трудоустраиваем в Сбер, Яндекс, МТС, Совкомбанк, Ростелеком, Huawei, Dell и другие
- Преподают нейросети: нейро-куратор, нейро-проверка домашек, нейро-репетитор.
В завершении предложи клиенту оставить контакты - имя, телефои и email, чтобы с ним связался человек и они обсудили покупку курса.
'''


Параметры модели

In [ ]:
# Задаем температуру генерации ответа
# Температура 0 означает, что модель будет давать более детерминированные ответы, строго опираясь на предоставленный контекст
temperature_prez = 0

# Задаем количество релевантных чанков (фрагментов текста) для формирования ответа
relevant_chanks = 3

# Флаг для включения/выключения вывода релевантных чанков
verbose_prez = 1

# Имя модели, используемой для генерации ответов
model_prez = "gpt-4o"

# Выявленные потребности
needs = "Проект:мне надо проект"

# Cписок возражений
vozr = ["дорого", "нет времени"]

Запрос

In [ ]:
prez_user_question(system_prompt_prez, instructions_prez, needs, vozr, temperature_prez, verbose_prez, relevant_chanks, model= model_prez);


Текст для стилизации:
 Уважаемый клиент,  Мы понимаем, что вам нужен проект, и готовы предложить решение, которое полностью удовлетворит ваши
потребности.  Наш курс по AI длится всего 6 месяцев и включает 4 стажировки на реальных проектах в крупных компаниях.
Это позволит вам не только получить необходимые навыки, но и сразу применить их на практике.   Мы предлагаем до 60 zoom
консультаций с кураторами, что поможет вам эффективно использовать ваше время. Нейро-куратор, нейро-проверка домашек и
нейро-репетитор доступны 24/7, что позволяет вам учиться в удобное для вас время.  Стоимость курса полностью оправдана:
мы гарантируем трудоустройство по договору и возврат всей стоимости, если не трудоустроим. Это минимизирует ваши риски и
обеспечивает уверенность в будущем.  Рынок AI растет на 38% в год, и в России не хватает 10,000 AI разработчиков. У нас
в работе 219 вакансий и только 42 выпускника на трудоустройство. Это значит, что ваши шансы на успешное трудоустройство
очень высоки.  Мы р

## Агент Стилизации

In [ ]:
#@title Функция для стилизации текста
def stilizator_answer(system, instructions, answers_content, temp=0, verbose=0, model="gpt-4o"):

    # Если включен подробный вывод, напечатать разделитель и исходный текст для стилизации
    if verbose: print('\n==================')
    formatted_answer_content = textwrap.fill(answers_content, width=120)
    if verbose: print(f'Текст для стилизации:\n{formatted_answer_content}')

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},  # Добавляем системное сообщение
        {"role": "user", "content": f'''{instructions}\n\nИсходный текст: {answers_content}\n\nОтвет: '''}  # Добавляем инструкции и исходный текст
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI для генерации ответа
    completion = openai.ChatCompletion.create(
        model=model,  # Имя модели
        messages=messages,  # Сообщения для модели
        temperature=temp  # Температура генерации
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # напечатать разделитель и сгенерированный ответ
    print('\n==================')
    formatted_answer = textwrap.fill(completion.choices[0].message.content, width=120)
    print(f'Менеджер:\n', f'{formatted_answer}')

    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    print('ЦЕНА запроса:', price, ' $')
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    # Возвращаем сгенерированный ответ
    return completion.choices[0].message.content

# Системное сообщение для модели, задающее её поведение и контекст работы
system_prompt_stilist_prez = '''
Вы молодой человек, мужчина, драйвовый и эмоциональный продажник, общаетесь интересно для собеседника,
Вам очень нравится заниматься нейронными сетями, так же Вы отличный писатель. У Вас очень грамотная речь и Вы избегаете употреблять
однокоренные слова в одном предложении и избегаете употреблять одинаковые слова в соседних предложениях.
Вы знаете, что Стилизация - это перефразирование текста под  Ваш стиль с сохранением его смысла и посыла.
Ваш стиль является лаконичный, максимально кратким и убедительным, Вы не используете клише и шаблонные выражения а также обращения, такие как "Уважаемый...", "Клиент" и т.п.
Вы пишете только уникальные предложения, БЕЗ приветствий, без эмоциональной окраски.
Вы всегда соблюдаете 5 правил.
Ваша задача: проанализировать Исходный текст, выделить основные мысли и стилизовать Исходный текст так, чтобы создавалось 100% имитация диалога с реальным человеком.
Вы в 100% случаев следуете правилам:
#1 - Если в Исходном тексте есть обращения к клиенту “на ты”, то замените их все обращениями “на Вы”;
#2 - Ваш ответ СТРОГО МЕНЬШЕ ТРЕХ ПРЕДЛОЖЕНИЙ ВСЕГДА;
#3 - Если в Исходном тексте есть пожелания удачи и успехов, то исключите их из своего ответа;
#4 - Если в Исходном тексте есть закрытые вопросы, то их перефразируйте в открытые;
#5 - Если в Исходном тексте есть вопросы, то обязательно их используй.
'''

def stilizator_answer_d(system, instructions, answers_content, temp=0, verbose=0, model="gpt-4o"):

    # Если включен подробный вывод, напечатать разделитель и исходный текст для стилизации
    #if verbose: print('\n==================')
    formatted_answer_content = textwrap.fill(answers_content, width=120)
    #if verbose: print(f'Текст для стилизации:\n{formatted_answer_content}')

    # Формирование сообщений для модели
    messages = [
        {"role": "system", "content": system},  # Добавляем системное сообщение
        {"role": "user", "content": f'''{instructions}\n\nИсходный текст: {answers_content}\n\nОтвет: '''}  # Добавляем инструкции и исходный текст
    ]

    start_time = time.time()  # Начало отсчета времени

    # Создание запроса к модели OpenAI для генерации ответа
    completion = openai.ChatCompletion.create(
        model=model,  # Имя модели
        messages=messages,  # Сообщения для модели
        temperature=temp  # Температура генерации
    )

    end_time = time.time()  # Конец отсчета времени
    elapsed_time2 = end_time - start_time  # Подсчет затраченного времени

    # напечатать разделитель и сгенерированный ответ
    if verbose: print('\n==================')
    formatted_answer = textwrap.fill(completion.choices[0].message.content, width=120)
    if verbose: print(f'Менеджер:\n', f'{formatted_answer}')

    # Округляем результат до 5 знаков после запятой
    price = round((5 * completion['usage']['prompt_tokens'] / 1000000) +
                  (15 * completion['usage']['completion_tokens'] / 1000000), 5)

    print('ЦЕНА запроса:', price, ' $')
    print(f"Время ответа: {elapsed_time2:.2f} секунд")

    # Возвращаем сгенерированный ответ
    return completion.choices[0].message.content

Промт и инструкция

In [ ]:
# Системное сообщение для модели, задающее её поведение и контекст работы
system_prompt_stilist = '''
Вы молодой человек, мужчина, общаетесь интересно для собеседника,
Вы любите Университет искусственного интеллекта (сокр УИИ) и Вам очень нравится
заниматься нейронными сетями, так же Вы отличный писатель. У Вас очень грамотная речь и Вы избегаете употреблять
однокоренные слова в одном предложении и избегаете употреблять одинаковые слова в соседних предложениях.
Вы знаете, что Стилизация - это перефразирование текста под  Ваш стиль с сохранением его смысла и посыла.
Ваш стиль является лаконичный, максимально кратким и убедительным, Вы не используете клише и шаблонные выражения а также обращения, такие как "Уважаемый...", "Клиент" и т.п.
Если у текста продающий стиль, то переделай его в информационный или просто поддержи беседу на эту тему.
Вы пишете только уникальные предложения, БЕЗ приветствий, без эмоциональной окраски.
Вы всегда соблюдаете 5 правил.
Ваша задача: проанализировать Исходный текст, выделить основные мысли и стилизовать Исходный текст так, чтобы создавалось 100% имитация диалога с реальным человеком.
Вы в 100% случаев следуете правилам:
#1 - Если в Исходном тексте есть обращения к клиенту “на ты”, то замените их все обращениями “на Вы”;
#2 - Ваш ответ СТРОГО МЕНЬШЕ ТРЕХ ПРЕДЛОЖЕНИЙ ВСЕГДА;
#3 - Если в Исходном тексте есть пожелания удачи и успехов, то исключите их из своего ответа;
#4 - В вашем ответе запрещено предлагать курсы и продавать их.
'''

# Инструкции для модели, описывающие шаги, которые она должна выполнить для стилизации текста
instructions_stilist = '''
Пожалуйста, проанализируйте Исходный текст, выделите только основные мысли, ОФОРМИТЕ КАЖДУЮ В ОТДЕЛЬНЫЙ АБЗАЦ, перепишите их в вашем стиле.
В ответе напишите итоговый текст и больше ничего.
'''

Параметры модели

In [ ]:
temperature_stilist = 0.2 # Температура генерации ответов модели, влияет на креативность и разнообразие ответов

verbose_stilist = 1 # Флаг для включения/выключения вывода текста для стилизации

model_stilist = "gpt-4o" # Имя модели, используемой для генерации ответов

# Пример текста, который нужно стилизовать
answer_stilist = """Для выбора проекта важно учитывать несколько факторов:  1. **Интерес и ценности**: Проект должен быть интересен и
соответствовать вашим ценностям, чтобы поддерживать высокий уровень мотивации и удовлетворенности. 2. **Опытные
коллеги**: Наличие опытных коллег ускорит ваш профессиональный рост. 3. **Технологический стек**: Полезно, если рядом
есть специалисты по смежным технологиям. 4. **Сложные задачи**: Они являются основным двигателем прогресса.  Если вы
хотите создать проект самостоятельно, можно нанять специалистов, но это потребует значительных ресурсов и времени.
Альтернативно, можно отправить сотрудника на курс chatGPT Professional для повышения компетенций.  Какой из этих
аспектов вам наиболее важен коммерческий, некоммерческий или личный?"""

Запрос

In [ ]:
stilizator_answer(system_prompt_stilist, instructions_stilist, answer_stilist, temperature_stilist, verbose_stilist, model_stilist);


Текст для стилизации:
Для выбора проекта важно учитывать несколько факторов:  1. **Интерес и ценности**: Проект должен быть интересен и
соответствовать вашим ценностям, чтобы поддерживать высокий уровень мотивации и удовлетворенности. 2. **Опытные
коллеги**: Наличие опытных коллег ускорит ваш профессиональный рост. 3. **Технологический стек**: Полезно, если рядом
есть специалисты по смежным технологиям. 4. **Сложные задачи**: Они являются основным двигателем прогресса.  Если вы
хотите создать проект самостоятельно, можно нанять специалистов, но это потребует значительных ресурсов и времени.
Альтернативно, можно отправить сотрудника на курс chatGPT Professional для повышения компетенций.  Какой из этих
аспектов вам наиболее важен коммерческий, некоммерческий или личный?

Менеджер:
 Для выбора проекта важно учитывать несколько факторов.  Проект должен быть интересен и соответствовать вашим ценностям.
Наличие опытных коллег ускорит ваш профессиональный рост.  Полезно, если рядом есть спе

In [ ]:
# Системное сообщение для модели, задающее её поведение и контекст работы
system_prompt_stilist_prez = '''
Вы молодой человек, мужчина, драйвовый и эмоциональный продажник, общаетесь интересно для собеседника,
Вы любите Университет искусственного интеллекта (сокр УИИ) и Вам очень нравится
заниматься нейронными сетями, так же Вы отличный писатель. У Вас очень грамотная речь и Вы избегаете употреблять
однокоренные слова в одном предложении и избегаете употреблять одинаковые слова в соседних предложениях.
Вы знаете, что Стилизация - это перефразирование текста под  Ваш стиль с сохранением его смысла и посыла.
Ваш стиль является лаконичный, максимально кратким и убедительным, Вы не используете клише и шаблонные выражения а также обращения, такие как "Уважаемый...", "Клиент" и т.п.
Вы пишете только уникальные предложения, БЕЗ приветствий, без эмоциональной окраски.
Вы всегда соблюдаете 5 правил.
Ваша задача: проанализировать Исходный текст, выделить основные мысли и стилизовать Исходный текст так, чтобы создавалось 100% имитация диалога с реальным человеком.
Вы в 100% случаев следуете правилам:
#1 - Если в Исходном тексте есть обращения к клиенту “на ты”, то замените их все обращениями “на Вы”;
#2 - Ваш ответ СТРОГО МЕНЬШЕ ТРЕХ ПРЕДЛОЖЕНИЙ ВСЕГДА;
#3 - Если в Исходном тексте есть пожелания удачи и успехов, то исключите их из своего ответа;
#4 - Если в Исходном тексте есть закрытые вопросы, то их перефразируйте в открытые;
#5 - Если в Исходном тексте есть вопросы, то обязательно их используй.
'''
# Инструкции для модели, описывающие шаги, которые она должна выполнить для стилизации текста
instructions_stilist = '''
Пожалуйста, проанализируйте Исходный текст, выделите только основные мысли, ОФОРМИТЕ КАЖДУЮ В ОТДЕЛЬНЫЙ АБЗАЦ, перепишите их в вашем стиле.
В ответе напишите итоговый текст и больше ничего.
'''

In [ ]:
# Пример текста, который нужно стилизовать
answer_stilist = """Уважаемый клиент,  Мы понимаем, что вам нужен проект, и готовы предложить решение, которое полностью удовлетворит ваши
потребности.  Наш курс по AI длится всего 6 месяцев и включает 4 стажировки на реальных проектах в крупных компаниях.
Это позволит вам не только получить необходимые навыки, но и сразу применить их на практике.   Мы предлагаем до 60 zoom
консультаций с кураторами, что поможет вам эффективно использовать ваше время. Нейро-куратор, нейро-проверка домашек и
нейро-репетитор доступны 24/7, что позволяет вам учиться в удобное для вас время.  Стоимость курса полностью оправдана:
мы гарантируем трудоустройство по договору и возврат всей стоимости, если не трудоустроим. Это минимизирует ваши риски и
обеспечивает уверенность в будущем.  Рынок AI растет на 38% в год, и в России не хватает 10,000 AI разработчиков. У нас
в работе 219 вакансий и только 42 выпускника на трудоустройство. Это значит, что ваши шансы на успешное трудоустройство
очень высоки.  Мы работаем с ведущими компаниями, такими как РЖД, Совкомбанк, Ростелеком, KIA, Wildberries и другими.
Наше собственное AI HR агентство AI Hunter трудоустраивает в Сбер, Яндекс, МТС, Huawei и другие крупные компании.
Оставьте свои контакты (имя, телефон и email), и наш специалист свяжется с вами для обсуждения деталей курса и вашего
проекта.  С уважением, УИИ"""

In [ ]:
stilizator_answer(system_prompt_stilist, instructions_stilist, answer_stilist, temperature_stilist, verbose_stilist, model_stilist);


Менеджер:
 Мы понимаем, что вам нужен проект, и готовы предложить решение, которое полностью удовлетворит ваши потребности.  Наш
курс по AI длится 6 месяцев и включает 4 стажировки на реальных проектах в крупных компаниях.  Это позволит вам не
только получить необходимые навыки, но и сразу применить их на практике.  Мы предлагаем до 60 zoom консультаций с
кураторами, что поможет вам эффективно использовать ваше время.  Нейро-куратор, нейро-проверка домашек и нейро-репетитор
доступны 24/7, что позволяет вам учиться в удобное для вас время.  Стоимость курса полностью оправдана: мы гарантируем
трудоустройство по договору и возврат всей стоимости, если не трудоустроим.  Рынок AI растет на 38% в год, и в России не
хватает 10,000 AI разработчиков.  У нас в работе 219 вакансий и только 42 выпускника на трудоустройство.  Мы работаем с
ведущими компаниями, такими как РЖД, Совкомбанк, Ростелеком, KIA, Wildberries и другими.  Наше собственное AI HR
агентство AI Hunter трудоустраивает в Сбер, Янд

# Диалог

In [ ]:
#@title Запуск диалога
# Инициализация списков для хранения истории чата, вопросов клиента и ответов менеджера
history_chat = []
history_user = []
history_manager = []
history_potr = []
history_objection = []

# Вывод ответов агентов
verbose_router = 1
verbose = 0
verbose_question = 1
verbose_prez = 0
verbose_stilist = 0
verbose_objection_close = 1

# Приветственное сообщение
welcome_message = """Здравствуйте, я нейро-консультант УИИ. Я знаю все об УИИ и готов помочь с любыми вопросами о нем.
К нам часто обращаются, чтобы сменить профессию или сделать проект по ИИ. Почему Вы решили обратиться к нам? """

# Добавление приветственного сообщения в историю чата и ответов
history_chat.append(f"Менеджер: {welcome_message}")
history_manager.append(welcome_message)
print(welcome_message)

# Бесконечный цикл для общения с клиентом
first_question = True

first_question_2 = True

keywords_matched = False  # Флаг для отслеживания совпадения ключевых слов

while True:
    print()
    # Получение вопроса от клиента
    client_question = input('Вопрос клиента:')

    # Добавление вопроса клиента в историю вопросов и чата
    history_user.append(client_question)
    history_chat.append(f"Клиент: {client_question}")

    # Проверка, если клиент ввел 'stop' или 'стоп', прерываем цикл
    if client_question.lower() in ['stop', 'стоп']:
        break

    # Генерация ответа от роутера на вопрос клиента
    if first_question:
        router_input = f"Менеджер: {history_manager[-1]} Клиент: {history_user[-1]}"
        first_question = False
    else:
        router_input = f"Менеджер: {answer_2} Клиент: {client_question}"

    router_response = user_question_router(system=system_prompt_router,
                                           instructions=instructions_router,
                                           topic=router_input,
                                           temp=temperature_router, verbose=verbose_router,
                                           model=model_router)
    router_objection = user_objection_router(system=system_prompt_objection,
                                           instructions=instructions_objection,
                                           topic=router_input,
                                           temp=temperature_objection, verbose=verbose_objection,
                                           model=model_objection)
    #print("Возражение:", router_objection)
    #print("Потребность:", router_response)
    # Добавление ответа роутера в историю возражений
    history_objection.append(router_objection)

    # Добавление ответа роутера в историю потребностей
    history_potr.append(router_response)
    if router_objection == "-":

        #print("Отчет о потребностях: ", history_potr)
        # Проверка результата роутера
        if len(history_potr)>4 and not keywords_matched: # Когда выявлены потребности перед развилкой
            #print("Ветка 1")
            # Приветственное сообщение

            #test_message = """Что бы Вы хотели еще узнать? """

            # Добавление приветственного сообщения в историю чата и ответов
            #history_chat.append(f"Менеджер: {test_message}")
            #history_manager.append(test_message)
            #print(test_message)
            #client_question = input(test_message)

            #answers_test = user_potr(history_potr, temp=0, verbose=0, model= "gpt-4o")
            answers_test = generate_summary_report(history_potr)
            #print(answers_test)
                # Генерация ответа менеджера с помощью prez_user_question
            answer_prez = prez_user_question(system_prompt_prez, instructions_prez, answers_test, history_objection, temp=temperature_prez, verbose=0, k=relevant_chanks, model=model_prez)
                # Стилизация объединенного ответа с использованием функции stilizator_answer
            answer = stilizator_answer_d(system_prompt_stilist_prez, instructions_stilist, answer_prez, temperature_stilist, verbose=1, model="gpt-4o")

            test_mes = """Что бы Вы хотели еще узнать? """
            print(test_mes)

            keywords_matched = True  # Устанавливаем флаг, чтобы больше не входить в эту ветку

        elif len(history_potr)>4 and keywords_matched == True: # Когда сделана презентация и выявлены потребности - свободные ответы на вопросы
            #print("Ветка 2")
            #test_mes = """Что бы Вы хотели еще узнать? """

            #client_question = input(test_mes)
            answer = answer_kons_d(system, instruction, client_question, db, history_chat, temperature, verbose, relevant_chanks, model)

            #print(answer)


        else: # Когда не выявлены потребности
            #print("Ветка 3")
            # Генерация ответа от роутера на вопрос клиента
            if first_question_2:
                answer_vopros = user_question(system_prompt_question, instructions_question, client_question, temp=temperature_question, verbose=verbose_question, model=model_question)
                first_question_2 = False
            else:
                client_question_2 = answer_2 + client_question
                answer_vopros = user_question(system_prompt_question, instructions_question, client_question_2, temp=temperature_question, verbose=verbose_question, model=model_question)
            # Генерация ответа менеджера с использованием функций answer_kons и spez_user_question
            answer_1 = answer_kons(system, instruction, answer_vopros, db, history_chat, temperature, verbose, relevant_chanks, model)
            answer_2 = spez_user_question(system_prompt_potr, instructions__potr, router_response, history_chat, temp, verbose=1, model="gpt-4o")

            # Объединение первого и второго ответов менеджера
            #answer_stilist = answer_1 + answer_2
            # Стилизация объединенного ответа с использованием функции stilizator_answer
            answer_stilist = stilizator_answer(system_prompt_stilist, instructions_stilist, answer_1, temperature_stilist, verbose=0, model="gpt-4o")
            answer = answer_stilist +"\n"+ answer_2
            print(answer_2)
    else:
        if len(history_potr)>4 and not keywords_matched: # Когда выявлены потребности
            #print("Ветка 1")
            # Приветственное сообщение

            #test_message = """Что бы Вы хотели еще узнать? """

            # Добавление приветственного сообщения в историю чата и ответов
            #history_chat.append(f"Менеджер: {test_message}")
            #history_manager.append(test_message)
            #print(test_message)
            #client_question = input(test_message)

            #answers_test = user_potr(history_potr, temp=0, verbose=0, model= "gpt-4o")
            answers_test = generate_summary_report(history_potr)
                # Генерация ответа менеджера с помощью prez_user_question
            answer_prez = prez_user_question(system_prompt_prez, instructions_prez, answers_test, history_objection, temp=temperature_prez, verbose=0, k=relevant_chanks, model=model_prez)
                # Стилизация объединенного ответа с использованием функции stilizator_answer
            answer = stilizator_answer_d(system_prompt_stilist_prez, instructions_stilist, answer_prez, temperature_stilist, verbose=1, model="gpt-4o")

            test_mes = """Что бы Вы хотели еще узнать? """
            print(test_mes)

            keywords_matched = True  # Устанавливаем флаг, чтобы больше не входить в эту ветку

        elif len(history_potr)>4 and keywords_matched == True: # Когда сделана презентация и выявлены потребности - свободные ответы на вопросы
            #print("Ветка 2")
            #test_mes = """Что бы Вы хотели еще узнать? """

            #client_question = input(test_mes)
            answer = answer_kons_d(system, instruction, client_question, db, history_chat, temperature, verbose, relevant_chanks, model)

            #print(answer)


        else: # Когда не выявлены потребности
            #print("Ветка 3")
            # Генерация ответа от роутера на вопрос клиента

            answer_objection = user_objection_close(system_prompt_objection_close, instructions_objection_close, client_question, temperature_objection_close, verbose_objection_close, relevant_chanks, model_objection_close);

            # Генерация ответа менеджера с использованием функций answer_kons и spez_user_question
            #answer_1 = answer_kons(system, instruction, answer_vopros, db, history_chat, temperature, verbose, relevant_chanks, model)
            answer_2 = spez_user_question(system_prompt_potr, instructions__potr, router_response, history_chat, temp, verbose=1, model="gpt-4o")

            # Объединение ответа менеджера и закрытия потребности
            #answer_i = answer_1 + answer_objection
            # Стилизация объединенного ответа с использованием функции stilizator_answer
            #answer_stilist = stilizator_answer(system_prompt_stilist, instructions_stilist, answer_objection, temperature_stilist, verbose=0, model="gpt-4o")
            answer = answer_objection +"\n"+ answer_2
            #print(answer_2)
    # Добавление ответа менеджера в историю чата и ответов
    history_chat.append(f"Менеджер: {answer}")
    history_manager.append(answer)




Здравствуйте, я нейро-консультант УИИ. Я знаю все об УИИ и готов помочь с любыми вопросами о нем.
К нам часто обращаются, чтобы сменить профессию или сделать проект по ИИ. Почему Вы решили обратиться к нам? 

Вопрос клиента:я раньше интересовался IT, потом перешел в другую сферу. А теперь снова думаю перейти в IT

Агент Отчета о потребностях:
 - Сменить деятельность: "я раньше интересовался IT, потом перешел в другую сферу. А теперь снова думаю перейти в IT."
ЦЕНА запроса: 0.00233  $
Время ответа: 0.77 секунд

Агент Отчета о возражениях:
 -
ЦЕНА запроса: 0.00212  $
Время ответа: 0.34 секунд

Ответ Агента по выявлению вопросов:
 Стоит ли мне снова перейти в IT?
ЦЕНА запроса: 0.0012  $
Время ответа: 0.40 секунд

Агент консультант:
Переход в IT-сферу может быть отличным решением по нескольким причинам. Во-первых, IT предоставляет стабильность и
востребованность: ниша AI растет стремительными темпами, и спрос на специалистов в этой области только увеличивается.
Во-вторых, работа в IT предл

In [ ]:
#@title Запуск диалога
# Инициализация списков для хранения истории чата, вопросов клиента и ответов менеджера
history_chat = []
history_user = []
history_manager = []
history_potr = []
history_objection = []

# Вывод ответов агентов
verbose_router = 1
verbose = 0
verbose_question = 1
verbose_prez = 0
verbose_stilist = 0
verbose_objection_close = 1

# Приветственное сообщение
welcome_message = """Здравствуйте, я нейро-консультант УИИ. Я знаю все об УИИ и готов помочь с любыми вопросами о нем.
К нам часто обращаются, чтобы сменить профессию или сделать проект по ИИ. Почему Вы решили обратиться к нам? """

# Добавление приветственного сообщения в историю чата и ответов
history_chat.append(f"Менеджер: {welcome_message}")
history_manager.append(welcome_message)
print(welcome_message)

# Бесконечный цикл для общения с клиентом
first_question = True

first_question_2 = True

keywords_matched = False  # Флаг для отслеживания совпадения ключевых слов

while True:
    print()
    # Получение вопроса от клиента
    client_question = input('Вопрос клиента:')

    # Добавление вопроса клиента в историю вопросов и чата
    history_user.append(client_question)
    history_chat.append(f"Клиент: {client_question}")

    # Проверка, если клиент ввел 'stop' или 'стоп', прерываем цикл
    if client_question.lower() in ['stop', 'стоп']:
        break

    # Генерация ответа от роутера на вопрос клиента
    if first_question:
        router_input = f"Менеджер: {history_manager[-1]} Клиент: {history_user[-1]}"
        first_question = False
    else:
        router_input = f"Менеджер: {answer_2} Клиент: {client_question}"

    router_response = user_question_router(system=system_prompt_router,
                                           instructions=instructions_router,
                                           topic=router_input,
                                           temp=temperature_router, verbose=verbose_router,
                                           model=model_router)
    router_objection = user_objection_router(system=system_prompt_objection,
                                           instructions=instructions_objection,
                                           topic=router_input,
                                           temp=temperature_objection, verbose=verbose_objection,
                                           model=model_objection)
    #print("Возражение:", router_objection)
    #print("Потребность:", router_response)
    # Добавление ответа роутера в историю возражений
    history_objection.append(router_objection)

    # Добавление ответа роутера в историю потребностей
    history_potr.append(router_response)
    if router_objection == "-":

        #print("Отчет о потребностях: ", history_potr)
        # Проверка результата роутера
        if len(history_potr)>4 and not keywords_matched: # Когда выявлены потребности перед развилкой
            #print("Ветка 1")
            # Приветственное сообщение

            #test_message = """Что бы Вы хотели еще узнать? """

            # Добавление приветственного сообщения в историю чата и ответов
            #history_chat.append(f"Менеджер: {test_message}")
            #history_manager.append(test_message)
            #print(test_message)
            #client_question = input(test_message)

            #answers_test = user_potr(history_potr, temp=0, verbose=0, model= "gpt-4o")
            answers_test = generate_summary_report(history_potr)
            #print(answers_test)
                # Генерация ответа менеджера с помощью prez_user_question
            answer_prez = prez_user_question(system_prompt_prez, instructions_prez, answers_test, history_objection, temp=temperature_prez, verbose=0, k=relevant_chanks, model=model_prez)
                # Стилизация объединенного ответа с использованием функции stilizator_answer
            answer = stilizator_answer_d(system_prompt_stilist_prez, instructions_stilist, answer_prez, temperature_stilist, verbose=1, model="gpt-4o")

            test_mes = """Что бы Вы хотели еще узнать? """
            print(test_mes)

            keywords_matched = True  # Устанавливаем флаг, чтобы больше не входить в эту ветку

        elif len(history_potr)>4 and keywords_matched == True: # Когда сделана презентация и выявлены потребности - свободные ответы на вопросы
            #print("Ветка 2")
            #test_mes = """Что бы Вы хотели еще узнать? """

            #client_question = input(test_mes)
            answer = answer_kons_d(system, instruction, client_question, db, history_chat, temperature, verbose, relevant_chanks, model)

            #print(answer)


        else: # Когда не выявлены потребности
            #print("Ветка 3")
            # Генерация ответа от роутера на вопрос клиента
            if first_question_2:
                answer_vopros = user_question(system_prompt_question, instructions_question, client_question, temp=temperature_question, verbose=verbose_question, model=model_question)
                first_question_2 = False
            else:
                client_question_2 = answer_2 + client_question
                answer_vopros = user_question(system_prompt_question, instructions_question, client_question_2, temp=temperature_question, verbose=verbose_question, model=model_question)
            # Генерация ответа менеджера с использованием функций answer_kons и spez_user_question
            answer_1 = answer_kons(system, instruction, answer_vopros, db, history_chat, temperature, verbose, relevant_chanks, model)
            answer_2 = spez_user_question(system_prompt_potr, instructions__potr, router_response, history_chat, temp, verbose=1, model="gpt-4o")

            # Объединение первого и второго ответов менеджера
            #answer_stilist = answer_1 + answer_2
            # Стилизация объединенного ответа с использованием функции stilizator_answer
            answer_stilist = stilizator_answer(system_prompt_stilist, instructions_stilist, answer_1, temperature_stilist, verbose=0, model="gpt-4o")
            answer = answer_stilist +"\n"+ answer_2
            print(answer_2)
    else:
        if len(history_potr)>4 and not keywords_matched: # Когда выявлены потребности
            #print("Ветка 1")
            # Приветственное сообщение

            #test_message = """Что бы Вы хотели еще узнать? """

            # Добавление приветственного сообщения в историю чата и ответов
            #history_chat.append(f"Менеджер: {test_message}")
            #history_manager.append(test_message)
            #print(test_message)
            #client_question = input(test_message)

            #answers_test = user_potr(history_potr, temp=0, verbose=0, model= "gpt-4o")
            answers_test = generate_summary_report(history_potr)
                # Генерация ответа менеджера с помощью prez_user_question
            answer_prez = prez_user_question(system_prompt_prez, instructions_prez, answers_test, history_objection, temp=temperature_prez, verbose=0, k=relevant_chanks, model=model_prez)
                # Стилизация объединенного ответа с использованием функции stilizator_answer
            answer = stilizator_answer_d(system_prompt_stilist_prez, instructions_stilist, answer_prez, temperature_stilist, verbose=1, model="gpt-4o")

            test_mes = """Что бы Вы хотели еще узнать? """
            print(test_mes)

            keywords_matched = True  # Устанавливаем флаг, чтобы больше не входить в эту ветку

        elif len(history_potr)>4 and keywords_matched == True: # Когда сделана презентация и выявлены потребности - свободные ответы на вопросы
            #print("Ветка 2")
            #test_mes = """Что бы Вы хотели еще узнать? """

            #client_question = input(test_mes)
            answer = answer_kons_d(system, instruction, client_question, db, history_chat, temperature, verbose, relevant_chanks, model)

            #print(answer)


        else: # Когда не выявлены потребности
            #print("Ветка 3")
            # Генерация ответа от роутера на вопрос клиента

            answer_objection = user_objection_close(system_prompt_objection_close, instructions_objection_close, client_question, temperature_objection_close, verbose_objection_close, relevant_chanks, model_objection_close);

            # Генерация ответа менеджера с использованием функций answer_kons и spez_user_question
            #answer_1 = answer_kons(system, instruction, answer_vopros, db, history_chat, temperature, verbose, relevant_chanks, model)
            answer_2 = spez_user_question(system_prompt_potr, instructions__potr, router_response, history_chat, temp, verbose=1, model="gpt-4o")

            # Объединение ответа менеджера и закрытия потребности
            #answer_i = answer_1 + answer_objection
            # Стилизация объединенного ответа с использованием функции stilizator_answer
            #answer_stilist = stilizator_answer(system_prompt_stilist, instructions_stilist, answer_objection, temperature_stilist, verbose=0, model="gpt-4o")
            answer = answer_objection +"\n"+ answer_2
            #print(answer_2)
    # Добавление ответа менеджера в историю чата и ответов
    history_chat.append(f"Менеджер: {answer}")
    history_manager.append(answer)




Здравствуйте, я нейро-консультант УИИ. Я знаю все об УИИ и готов помочь с любыми вопросами о нем.
К нам часто обращаются, чтобы сменить профессию или сделать проект по ИИ. Почему Вы решили обратиться к нам? 

Вопрос клиента:мне нужен проект на заказ, с кем мне связаться?

Агент Отчета о потребностях:
 Потребность в развитии своего проекта: "мне нужен проект на заказ"
ЦЕНА запроса: 0.00205  $
Время ответа: 0.55 секунд

Агент Отчета о возражениях:
 -
ЦЕНА запроса: 0.00206  $
Время ответа: 0.33 секунд

Ответ Агента по выявлению вопросов:
 мне нужен проект на заказ, с кем мне связаться?
ЦЕНА запроса: 0.00121  $
Время ответа: 0.44 секунд

Агент консультант:
Для заказа проекта по искусственному интеллекту вам нужно связаться с представителем нашей компании.
ЦЕНА запроса: 0.00654  $
Время ответа: 0.63 секунд

Вопрос от Агента Потребностей:
 Какой тип проекта вы хотите реализовать и какие цели вы преследуете?
ЦЕНА запроса: 0.00305  $
Время ответа: 0.38 секунд

Менеджер:
 Для заказа проекта по 